### SQLite資料庫
* 不需要安裝額外的資料庫管理程式或伺服器，只要一個檔案就可以使用的資料庫系統SQLite，是初學者和小型資料庫應用程式的最佳選擇，儘管嚴格來說它其實還不算是一個資料庫管理系統。
* SQLite是一個以檔案為基礎的非常精簡版的SQL資料庫管理系統，它的最主要特色是沒有外部的伺服器系統或是執行中的對應程式，所有的操作都內嵌在操作資料庫的應用程式中（也就是我們編寫的程式），對於Python來說，只要載入對應的模組即可立即上手使用。

In [ ]:
#建立資料庫
import sqlite3
con = sqlite3.connect('mydatabase.db')

In [ ]:
#建立資料庫Cursor變數 (Cursor是系統為使用者開設的一個數據緩衝區，存放SQL語句的執行結果)
con = sqlite3.connect('mydatabase.db')
cursorObj = con.cursor()

In [ ]:
#建立Table
employees (id, name, salary, department, position, hireDate)
#使用 DB browser for sqlite 檢查資料是否建立成功

In [ ]:
#插入一筆資料到Table
cursorObj.execute("INSERT INTO employees VALUES(1, 'John', 700, 'HR', 'Manager', '2017-01-04')")
con.commit()
#VALUES裡面的值，可用?代替，表示缺失的空值

In [ ]:
#更新資料到Table
import sqlite3
con = sqlite3.connect('mydatabase.db')

def sql_update(con):
    cursorObj = con.cursor()
    cursorObj.execute('UPDATE employees SET name = "Rogers" where id = 2')
    con.commit()
sql_update(con)

In [ ]:
#選擇資料
cursorObj.execute('SELECT * FROM employees ')

In [ ]:
#取得所有資料
print(row) for row in cursorObj.fetchall()

In [ ]:
#取得指定資料
import sqlite3
con = sqlite3.connect('mydatabase.db')

def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT id, name FROM employees WHERE salary > 800.0')
    rows = cursorObj.fetchall()
    for row in rows:
        print(row)
sql_fetch(con)

In [ ]:
#顯示有多少ROW在上一次操作時受到影響
rows = cursorObj.fetchall()
print len (rows)
print(cursorObj.execute('DELETE FROM employees').rowcount)

In [ ]:
#列出所有Tables
import sqlite3
con = sqlite3.connect('mydatabase.db')
def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    print(cursorObj.fetchall())
    
sql_fetch(con)

In [ ]:
#刪除Table
import sqlite3
 
con = sqlite3.connect('mydatabase.db')
 
def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('DROP table if exists employees')
    con.commit()
    
sql_fetch(con)

In [ ]:
#插入大量資料
import sqlite3
 
con = sqlite3.connect('mydatabase.db')
 
cursorObj = con.cursor()
 
cursorObj.execute('create table if not exists projects(id integer, name text)')
 
data = [(1, "Ridesharing"), (2, "Water Purifying"), (3, "Forensics"), (4, "Botany")]
 
cursorObj.executemany("INSERT INTO projects VALUES(?, ?)", data)
 
con.commit()

In [ ]:
#關閉與資料庫的連接
con = sqlite3.connect('mydatabase.db')
con.close()

In [ ]:
#建立時間資料
import sqlite3
import datetime
con = sqlite3.connect('mydatabase.db')
cursorObj = con.cursor()
cursorObj.execute('create table if not exists assignments(id integer, name text, date date)')
data = [(1, "Ridesharing", datetime.date(2017, 1, 2)), (2, "Water Purifying", datetime.date(2018, 3, 4))]
cursorObj.executemany("INSERT INTO assignments VALUES(?, ?, ?)", data)
con.commit()

In [ ]:
#顯示學生成績表
import sqlite3
dbfile = "school.db"
#使用sqlite3.connect("資料庫檔案路徑")，它會傳回一個指標
conn = sqlite3.connect(dbfile)
#利用前面所傳回的變數conn呼叫execute函數，函數中的內容就是SQL指令的字串。
#在execute執行之後，會傳回一個叫做Cursor的物件，它可以用來存取每一筆紀錄，也就是資料表中查詢結果的指標，由於會以列的型式來存取
#接著，可以使用迴圈的方式把每一列找出來使用。
rows = conn.execute("select * from score;")
for row in rows:
    for field in row:
        print("{}\t".format(field), end="")
    print()
conn.close()

In [ ]:
#輸入成績並把它們儲存入資料庫的標準作法
import sqlite3
dbfile = "school.db"
conn = sqlite3.connect(dbfile)
stuno = input("學號：")
chi = input("國文成績：")
eng = input("英文成績：")
mat = input("數學成績：")
his = input("歷史成績：")
geo = input("地理成績：")
sql_str = "insert into score(stuno, chi, eng, mat, his, geo) values('{}',{},{},{},{},{});".format(stuno, chi, eng, mat, his, geo)
conn.execute(sql_str)
conn.commit()
conn.close()

In [ ]:
#設定以欄位名稱操作資料庫的程式設計標準步驟
mport sqlite3
dbfile = "school.db"
conn = sqlite3.connect(dbfile)
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute("select * from score;")
rows = cur.fetchall()
print(rows[0].keys())
print(type(rows))
print(type(rows[0]))
print("學號\t國文\t英文")
for row in rows:
    print("{}\t{}\t{}".format(row['stuno'], row['chi'], row['eng']))

### 練習
1. 讀json檔得到資料
2. 資料整理成方便取值的樣子
3. 判斷多筆輸入或單筆輸入
4. 將資料按照格式填入資料庫中

In [ ]:
import sqlite3
import os
import json

class DBConnector:
    def __init__(self, database):
        self.connector = None
        self.database = database
        self.get_connection(database)

    def get_connection(self, database=None):
        if database or (self.connector is None):
            self.connector = sqlite3.connect(database)
            self.connector.row_factory = DBConnector.dict_factory   # 印出第一欄
            self.connector.execute("PRAGMA foreign_keys = ON")    # 可使用pk
        return self.connector

    def close_connection(self):
        if self.connector is not None:
            self.connector.close()

    def remove_db(self):
        if os.path.isfile(self.database):
            os.remove(self.database)

    @staticmethod
    def dict_factory(cursor, row):
        return {column[0]: row[index] for index, column in enumerate(cursor.description)}


def insert_data(table_name, data_file, multiple_insert):
    database = "./student_score.db"
    db_connector = DBConnector(database)
    connection = db_connector.get_connection()
    cursor = connection.cursor()
    insert_string = []
    multiple_out = ()

    with open(data_file, 'r') as f:
        datas = json.loads(f.read())

    for data in datas:
        student = data[0]
        subject_score = data[1]

        subject_number = len(subject_score)  # subject_number = 10
        len1 = len(subject_score[0])    # len1 = 2

        if (multiple_insert == True):    # 多筆輸入
            insert_string = '''INSERT INTO {} VALUES'''.format(table_name)
            for i in range(subject_number):
                for j in range(0, len1, 2):
                    multiple_out = (student, subject_score[i][j], subject_score[i][j + 1])
                    if i == (subject_number-1):
                        insert_string += ("{}".format(multiple_out))
                    else:
                        insert_string += ("{},".format(multiple_out))

            insert_string += ";"
            cursor.execute(insert_string)
            connection.commit()

        if(multiple_insert == False):  # 單筆輸入
            for i in range(subject_number):
                for j in range(0, len1, 2):
                    insert_string = '''INSERT INTO {} VALUES ('{}', '{}', {});'''.format(table_name,student,subject_score[i][j],subject_score[i][j + 1])
                    cursor.execute(insert_string)
            connection.commit()
    return

if __name__ == "__main__":  
    insert_data("score", "dict_test2.json", multiple_insert=True)

mutiple_insert=True: (多筆輸入)
```
INSERT INTO {"table_name"} VALUES ('{student name}', '{subject 1}', {score}),
                                    ('{student name}', '{subject 2}', {score}),
									('{student name}', '{subject 3}', {score}),
											.....
                                  ('{student name}', '{subject n}', {score});
```

mutiple_insert=False: (單筆輸入)
```
INSERT INTO {"table_name"} VALUES ('{student name}', '{subject 1}', {score});
INSERT INTO {"table_name"} VALUES ('{student name}', '{subject 2}', {score});
INSERT INTO {"table_name"} VALUES ('{student name}', '{subject 3}', {score});
```